In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time;  # 引入time模块
import requests

maxPage = 7    #遍历的页数区间
nowPage = 0
mySno = "你的学号"
rootUrl = "http://jsxy.zucc.edu.cn"
chrome_options=Options()
#设置chrome浏览器无界面模式
#chrome_options.add_argument('--headless')
#设置代理
#PROXY = "x.x.x.x:1080";
# Create a copy of desired capabilities object.
#desired_capabilities = webdriver.DesiredCapabilities.INTERNETEXPLORER.copy()
# Change the proxy properties of that copy.
# desired_capabilities['proxy'] = {
#     "httpProxy":PROXY,
#     "ftpProxy":PROXY,
#     "sslProxy":PROXY,
#     "noProxy":None,
#     "proxyType":"MANUAL",
#     "class":"org.openqa.selenium.Proxy",
#     "autodetect":False
# }
#建立浏览器实例
browser = webdriver.Chrome()
# browser = webdriver.Chrome(chrome_options=chrome_options,desired_capabilities =desired_capabilities )
# 开始请求
browser.implicitly_wait(10) # seconds




In [15]:
def go_to_erke_page(browser):    
    alinks = browser.find_elements_by_tag_name('a')   #在官网首页找到所有<a>标签
    for x in alinks:
        if x.get_attribute('innerHTML')=='第二课堂':  #在找到标签中是“第二课堂”的
            root_url = x.get_attribute('href')        #获取超链接
    browser.get(root_url)                             #跳转到第二课堂页面

In [16]:
def get_all_link_in_maindiv(browser):                 #列表解析，返回第二课堂列表页面每条标题和超链接
    maindiv = browser.find_element_by_class_name("default_pgContainer")
    return [{'title':x.get_attribute('innerHTML'),'src':x.get_attribute('href')} for x in maindiv.find_elements_by_xpath(".//a")]

In [17]:
def go_to_next_page(browser):                         #跳转到下一页
    alinks = browser.find_elements_by_tag_name('a')
    nextPage = filter(lambda alink:alink.get_attribute('title').find('下页')!=-1, alinks)
    nextPage = list(nextPage)[0]
    while( True ):
        try:
            nextPage.click()
            break;
        except:
            print('watting btn clickable..')
            time.sleep(1)
            continue

In [18]:
def find_xls_files(browser):      #找到页面中的文件，返回文件名和超链接列表
    alinks = browser.find_elements_by_tag_name('a')
    xlsdownloads = filter(lambda alink:alink.get_attribute('href').find('.xls')!=-1, alinks)
    return [{'filename':x.get_attribute('text'),'src':x.get_attribute('href')} for x in xlsdownloads]

In [19]:
browser.get(rootUrl)
xlsfilesSRC = []


go_to_erke_page(browser)
while nowPage<maxPage:
    nowPage = nowPage + 1
    pages = get_all_link_in_maindiv(browser)
    currentUrl = browser.current_url;
    for page in pages:
        browser.get(page['src'])
        xlsfilesSRC = xlsfilesSRC + find_xls_files(browser)     #找出各个页面的文件超链接存入列表中
    browser.get(currentUrl)
    go_to_next_page(browser)

watting btn clickable..
watting btn clickable..
watting btn clickable..
watting btn clickable..


In [20]:
tmpfilepath = 'tmp.xls'
def download_to_tmpfile(downloadurl):
    r = requests.get(downloadurl) 
    with open(tmpfilepath,'wb') as f:  #打开一个tmp.xls文件用于写入下载下来的文件内容
        f.write(r.content)
    f.close()


In [21]:
import xlrd  #读excel的库





In [22]:
def find_rows_in_xls(targetStr):         
    data = xlrd.open_workbook(tmpfilepath)  #在Excel中找到目标行，并返回
    table = data.sheets()[0] 
    findRows = []
    for row in table.get_rows():
        findCells = filter( lambda cell:str(cell.value).find(targetStr)!=-1, row)
        if len(list(findCells)) > 0:
            findRows.append(row)
    return findRows;


In [23]:
found = []
notFound = []

for file in xlsfilesSRC:
    src = file['src']
    download_to_tmpfile(src)
    findRow = find_rows_in_xls(mySno)
    if len(findRow) > 0 :
        found.append({'file':file,'rows':findRow[0]}) # TODO 一个文件中找到多个
    else:
        notFound.append(file)

In [24]:
len(found)
import xlwt  #写excel的库

outFile=xlwt.Workbook()                       #将找到的内容写入Excel文件
sheet=outFile.add_sheet("sheet2")  
sheetTitle = ["文件名",'下载路径','合成内容','内容']
[sheet.write(0,index,content) for index,content in enumerate(sheetTitle,0)]
for row,rec in enumerate(found,1):
    sheet.write(row,0,rec['file']['filename'])
    sheet.write(row,1,rec['file']['src'])
    sheet.write(row,2, " ".join(list(map(lambda cell:str(cell.value),rec['rows']))) ) # TODO number类型的cell会带小数点
    for column,text in enumerate(rec['rows'],3):
        sheet.write(row,column,text.value)
outFile.save(mySno+"找到的.xls") 
